In [5]:
import pandas as pd
import networkx as nx
import numpy as np
from ipysigma import Sigma
from IPython.display import display
import ipywidgets as widgets

# Load Excel file
file_path = "https://github.com/MitaliPattaniSFU/Course_Requisites/raw/main/Departments/BPK.xlsx"
df = pd.read_excel(file_path, engine='openpyxl')

# Department positions and colors
dept_positions = {
    "BPK": (0, 0),
    "BISC": (3, 0),
    "PHYS": (6, 0),
    "MBB": (9, 0),
    "CHEM": (12,0),
    "PSYC": (15, 0),
    "EDUC": (18,0),
    "HLTHSCI": (21,0),
    "STAT": (24,0),
    "MATH": (27,0)
}
dept_colors = {
    "BPK": "#62a0ca", # blue
    "BISC": "#ff7f0e", # orange
    "PHYS": "#495e35", # dark olive green
    "MBB": "#bfd1ae", # pale green
    "CHEM": "#F5B1A4",  # light pink
    "PSYC": "#86a865", # medium green
    "EDUC": "#c29ba3",  # muted rose
    "HLTHSCI": "#7b9ba6", # desaturated teal
    "STAT": "#9467bd",   # purple
    "MATH": "#c5b0d5" #light lavender
}

def random_offset(center, radius=3):
    angle = np.random.uniform(0, 2 * np.pi)
    r = np.random.uniform(0, radius)
    return center[0] + r * np.cos(angle), center[1] + r * np.sin(angle)

# Function to build the graph with optional highlighting
def build_graph(highlight_node=None):
    G = nx.DiGraph()
    for _, row in df.iterrows():
        course = row['Course_Code']
        prerequisites = row['Prerequisites']
        corequisites = row.get('CoRequisites', None)
        node_type = row['Type']
        node_dept = str(row['Dept']).strip().upper()
        description = row['Description']

        color = dept_colors.get(node_dept, "#7f7f7f")
        x, y = random_offset(dept_positions.get(node_dept, (20, 20)))
        #print(course, '->', node_dept, '->', color)
        G.add_node(course, label=course, Dept=node_dept,
                   Prerequisites=prerequisites, Corequisites=corequisites,
                   Description=description, color=color, x=x, y=y)

    # Add edges after all nodes are added
    for _, row in df.iterrows():
        course = row['Course_Code']
        prerequisites = row['Prerequisites']
        corequisites = row.get('CoRequisites', None)
        x, y = G.nodes[course]['x'], G.nodes[course]['y']

        if pd.notna(prerequisites) and prerequisites != 'none':
            for prereq in prerequisites.split(","):
                prereq = prereq.strip()
                if not G.has_node(prereq):
                    G.add_node(prereq, label=prereq, color=color, x=x + 0.5, y=y + 0.5)  
                G.add_edge(prereq, course, type='arrow')

        if pd.notna(corequisites) and corequisites != 'none':
            for coreq in corequisites.split(","):
                coreq = coreq.strip()
                if not G.has_node(coreq):
                    G.add_node(coreq, label=coreq, color=color, x=x + 0.5, y=y + 0.5)
                G.add_edge(coreq, course, type='arrow', color='#00b894')

    return G


# Initial graph
G = build_graph()
sigma = Sigma(
    G,
    node_color="Dept",
    node_color_palette=dept_colors,
    node_label="label",
    node_label_size=G.degree,
    node_size=G.degree,
    start_layout=True
)

display(sigma)


Sigma(nx.DiGraph with 143 nodes and 286 edges)

In [6]:
# Export the graph to an HTML file
sigma.write_html(
    G,
    r"C:\Users\mpattani\Documents\Course_Requisites\Visulizations\BPK.html",
    fullscreen=True,
    node_color="Dept",
    node_color_palette=dept_colors,
    node_size_range=(1, 30),
    node_border_color_from='node',
    default_node_label_size=12,
    edge_size_range=(1, 30),
    node_size=lambda node: G.degree(node)
)